In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.parse import quote
from matplotlib import pyplot as plt
from sys import platform
import bs4, ssl
import pandas as pd
import re
import datetime, time # for sleep
import requests
import numpy as np
import io
import codecs

import xml.etree.ElementTree as et 

import json
import urllib
import urllib.request
import sys

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                            'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                            'Chrome/23.0.1271.64 Safari/537.11', \
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
            'Accept-Encoding': 'none', \
            'Accept-Language': 'en-US,en;q=0.8', \
            'Connection': 'keep-alive'}

context = ssl._create_unverified_context()

############################################################
def __Last_Page_Find(i, Market) :
    # print("# Last_Page_Find Start")
    if Market == 1 :
        url = "https://finance.naver.com/sise/sise_market_sum.nhn?&page=" + str(i)
    else :
        url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page=" + str(i)
    # print("# print URL")
    # print(url) # ---> 당시의 위치정보를 기반으로 15페이지 가져오는 듯, 어찌 가져오는지는 모르겠음...(2020.05.21)
    # ---> HTTP Error 403: Forbidden
    # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    # req = Request(url=url, headers={'User-Agent': 'Mozilla/5.0', 'referer': 'http://m.naver.com'})

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                             'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                             'Chrome/23.0.1271.64 Safari/537.11', \
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
               'Accept-Encoding': 'none', \
               'Accept-Language': 'en-US,en;q=0.8', \
               'Connection': 'keep-alive'}
    req = Request(url=url, headers=headers)

    try :
        html = urlopen(req).read()
        # html = urlopen(url, context=context)
    except HTTPError as err:
        if err.code == 404 :
            print("# Error HTTP : 404")
            return
        elif err.code == 403 :
            print("# Error HTTP : 403")
            return
        elif err.code == 500 :
            print("# Error HTTP : 500")
            return
        elif err.code == 505:
            print("# Error HTTP : 505")
            return
        else:
            raise

    bs_object = bs4.BeautifulSoup(html, "html.parser")
    # test = bs_object.find('table' ,attrs={'class':'type_2'})
    # numbersss = test.findAll("td", {"class": "number"})
    # print(numbersss)

    data = []
    table = bs_object.find('table', attrs={'class':'type_2'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    # print("##################")
    # for x in data :
        # print(x)

    # print("##########################################")
    # print(type(data))
    Data_DF = pd.DataFrame(data, columns=['No','회사명','현재가','전일비','등락률','액면가','시가총액','상장주식수','외국인비율','거래량','PER','ROE'])
    # print(type(data_df))
    # print(data_df)
    # print("# Drop NA")
    Data_DF = Data_DF.dropna()
    Data_DF = Data_DF[['No','회사명','ROE','PER','외국인비율','현재가']]
    if Market == 1 :
        Data_DF['Market'] = "코스피"
    else :
        Data_DF['Market'] = "코스닥"

    return Data_DF

############################################################
def __Last_Page_Numb(company_code, dist) :

    if dist == 1 :
        url = "https://finance.naver.com/item/sise_day.nhn?code=" + company_code
    else :
        url = "https://finance.naver.com/research/company_list.nhn?searchType=itemCode&itemCode=" + company_code
    
    print(url)

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
        'AppleWebKit/537.11 (KHTML, like Gecko) ' \
        'Chrome/23.0.1271.64 Safari/537.11', \
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
        'Accept-Encoding': 'none', \
        'Accept-Language': 'en-US,en;q=0.8', \
        'Connection': 'keep-alive'}

    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    bs_object = bs4.BeautifulSoup(html, "html.parser")
    table = bs_object.find('td', attrs={'class':'pgRR'})
    if table is None :
        print("# Naver Report 없음")
        return 10000

    print(table)

    for x in table.find_all('a') :
        test = x['href']
        active = test.split("=")
        if dist == 1 :
            page_num = active[2]
        else :
            page_num = active[3]

    if dist == 1 :
        print("# Company Page_Number : ", page_num)
    else :
        print("# Report Page_Number : ", page_num)

    return page_num
############################################################
def __Last_Page_Numb_Market(Market) :
    if Market == "KOSPI" :
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=1"
        # print(url)
    elif Market == "KOSDAQ" :
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSDAQ&page=1"
        # print(url)
    print("Market : %s" % url)

    html = urlopen(url, context=context)
    bs_object = bs4.BeautifulSoup(html, "html.parser")
    main1_pages = bs_object.find_all("a")
    for a_list in main1_pages:
        if a_list is main1_pages[-1] :
            last_page=a_list.attrs['href']
        else :
            continue
        page_num: object=last_page.split('=')
        print("# Market Page_Number : ", page_num[2])
        return page_num[2]

############################################################
def __Price_Find(company_code, page):
    page = int(page)
    df_list, date_list, closing_list, gap_list, gap_perf  = [], [], [], [], []

    date_list, closing_list = [], []
    for page_num in range(1, page) :
    # for page_num in range(1, 125) : # ---> 1페이지 당 10일, 250일이 년중 평일이라 하면 25페이지가 1년, 125페이지가 5년.
        url= "https://finance.naver.com/item/sise_day.nhn?code=" + company_code + "&page=" + str(page_num)
        print(url)

        req = Request(url=url, headers=headers)

        try :
            html = urlopen(req).read()
            # html = urlopen(url, context=context)
        except HTTPError as err:
            if err.code == 404 :
                print("# Error HTTP : 404")
                return
            elif err.code == 403 :
                print("# Error HTTP : 403")
                return
            elif err.code == 500 :
                print("# Error HTTP : 500")
                return
            elif err.code == 505:
                print("# Error HTTP : 505")
                return
            else:
                raise

        bs_object = bs4.BeautifulSoup(html, "html.parser")
        table = bs_object.find('table', attrs={'class':'type2'})
        table_body = table.find_all('td', attrs={'class':'num'})
        date_body = table.find_all('span', attrs={'class':'tah p10 gray03'})

        for x in date_body :
            # print("x.contents type : %s" % type(x.contents)) # ---> x.contents type : <class 'list'>
            date_list.append(x.contents[0])

        testlist = []
        for tr in table_body :
            td = tr.find_all('span', attrs={'class':'tah p11'})
            # print("for tr type : %s" % type(td))
            for x in td :
                # print("for x in td : %s" % type(x)) # ---> for tr type : <class 'bs4.element.ResultSet'>
                # print("for x.contents in td : %s, %s" % (type(x.contents), x.contents)) # ---> for x.contents in td : <class 'list'>
                # print("for x.contents in td : %s, %s" % (type(x.contents[0]), x.contents[0])) # ---> for x.contents in td : <class 'list'>
                # time.sleep(3)
                # ----> 테이블 중간에 전일비 값이 0이면 table이 엉망이 됨
                # ----> 전일비가 안들어오다가 들어간o
                # print("X Contennts : %s, %s" % (x.contents[0], type(x.contents[0])))
                # print("X Contennts : %s, %s" % (x.contents[0], type(str(x.contents[0]))))
                if str(x.contents[0]) == "0" :
                    continue
                else :
                    testlist.append(x.contents[0])
            
        # print("# TEST LIST")
        # print(testlist)

        for x in range(0, len(testlist)) :
            # print(x, testlist[x])
            if x == 0 :
                closing_list.append(testlist[x])
            else :
                result = x % 5
                if result == 0 :
                    closing_list.append(testlist[x])

    # print("############## date_list ##################")
    # print(date_list)
    # print("############## closing_list ##################")
    # print(closing_list)

    return date_list, closing_list

############################################################
def __Market_Price(company_code, page):
    df_list, date_list, closing_list = [], [], []
    # print("# Market : %s" % company_code)

    print("# Company : %s" % company_code)
    page = int(page)
    for page_num in range(1, page) :
    # for page_num in range(1, 100) :
        url_main = "https://finance.naver.com/sise/sise_index_day.nhn?code=" + company_code + "&page=" + str(page_num)
        # print("# url : %s" % url_main)
        code_data = pd.read_html(url_main, header=0, skiprows=(6,7,8)) # ---> code data로 출력을 생성하면 NaN이 1개라인이 남음, Type은 <class 'list'>
        df_list.append(pd.read_html(url_main)[0]) # ---> List 로 append, 본문[0]과 페이지 출력[1] 부분 중 본문을 출력

    df=pd.concat(df_list)
    df.dropna(subset=["날짜"], inplace=True) # ---> Naver에서 출력되는 가운데 날짜 컬럼 NaN 라인들 삭제처리
    df.rename(columns={'날짜': 'date'}, inplace = True)
    df.rename(columns={'체결가': 'closing'}, inplace = True)
    df.rename(columns={'전일비': 'diff'}, inplace = True)
    df['new_date'] = pd.to_datetime(df['date']) # ---> dateime 형식으로 new_date 추가
    date_list=df.date.values.tolist()
    closing_list=df.closing.values.tolist()
    for x in range(0, len(closing_list)-1):
        closing_list[x] = int(closing_list[x]) # ---> float 에서 int로 변경

    # date_list.reverse()
    # closing_list.reverse()

    return date_list, closing_list

############################################################
def __New_Make_Chart (Dart_Exp_Code, Dart_Comp_Name, Market, Market_Date, Market_List) :

    DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")

    if platform == 'darwin' : # or win32
        DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")
        File_Name = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/"+Dart_Comp_Name+".html"
        input_1 = open('/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/New_Echart_Test_1.html', 'r', encoding='utf-8') # Echrt의 머릿글
        output = open(File_Name, 'w+t')
    else :
        DateTime_Win = datetime.datetime.today().strftime("%Y%m%d")
        File_Name = Dart_Comp_Name+".html"
        input_1 = open('New_Echart_Test_1.html', 'r', encoding='utf-8') # Echrt의 머릿글
        output = open(File_Name, 'w+t', encoding='utf-8')

    # 아래는 Make_Chart를 위한 조건들
    Comp_Date_List, KOSPI_DATE_List, KOSDAQ_DATE_List = [], [], []
    Comp_Closing_List, KOSPI_List, KOSDAQ_List = [], [], []

    Exp_page = __Last_Page_Numb(Dart_Exp_Code, 1)
    Comp_Date_List, Comp_Closing_List = __Price_Find(Dart_Exp_Code, Exp_page)
    date_list = Comp_Date_List

    print("Market : %s" % Market)

    # if Market == "코스피" :
    #     KOSPI_Page = __Last_Page_Numb_Market("KOSPI")
    #     KOSPI_Date, KOSPI_List = __Market_Price("KOSPI", KOSPI_Page)
    #     closing_list_A = KOSPI_List
    #     closing_list_B = Comp_Closing_List
    # elif Market == "코스닥" :
    #     KOSDAQ_Page = __Last_Page_Numb_Market("KOSDAQ")
    #     KOSDAQ_Date, KOSDAQ_List = __Market_Price("KOSDAQ", KOSDAQ_Page)
    #     closing_list_A = KOSDAQ_List
    #     closing_list_B = Comp_Closing_List

    closing_list_A = Market_List
    closing_list_B = Comp_Closing_List

    n = len(closing_list_B)
    print("date list 0 : %s" % date_list[0])
    print("date leng : %s" % len(date_list))
    print("closing list A-0 : %s " % closing_list_A[0])
    print("closing leng A : %s " % len(closing_list_A))
    print("closong list B-0 : %s " % closing_list_B[0])
    print("closing leng B : %s " % len(closing_list_B))
    del date_list[n:]
    del closing_list_A[n:]
    del closing_list_B[n:]
    print("# After DEL")
    print("date list 0 : %s" % date_list[0])
    print("closing list A-0 : %s " % closing_list_A[0])
    print("closing leng A : %s " % len(closing_list_A))
    print("closong list B-0 : %s " % closing_list_B[0])
    print("closing leng B : %s " % len(closing_list_B))

    # ls = [type(item) for item in closing_list_A]
    # print(ls) # ---> int
    # ls = [type(item) for item in closing_list_B]
    # print(ls) # ---> <class 'bs4.element.NavigableString'>
    # 천 단위 , 자리수가 있으면 string이고 없으면 int 로 list 생성되는듯..

    for idx, val in enumerate(closing_list_A) :
        if type(closing_list_A[idx]) == int or type(closing_list_A[idx]) == float :
            continue
        else :
            val = val.replace(",","")
            closing_list_A[idx] = val

    for idx, val in enumerate(closing_list_B) :
        if type(closing_list_B[idx]) == int or type(closing_list_B[idx]) == float :
            continue
        else :
            val = val.replace(",","")
            closing_list_B[idx] = val

    # print(date_list.reverse()) # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x12235bbb0>
    # date_list.reverse() # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x1190a7d00>
    # print(date_list[::-1]) # ---> OK

    # max_value_A = int(max(closing_list_A)) * 2
    # max_value_B = int(max(closing_list_B)) * 2
    numrow_A = [float(x) for x in closing_list_A]
    max_value_A = int(max(numrow_A)) * 1.5
    numrow_B = [float(x) for x in closing_list_B]
    max_value_B = int(max(numrow_B)) * 1.5 # ---> ValueError: could not convert string to float: '11,550'

    print("# MAKE HTML Start")
    for line in input_1.readlines():
        output.write(line)
    output.write("        data: ['%s', '%s']\n" % (Market, Dart_Comp_Name))
    output.write("    },\n")
    output.write("    xAxis: [\n")
    output.write("        {\n")
    output.write("            type: 'category',\n")
    output.write("            axisTick: {\n")
    output.write("                alignWithLabel: true\n")
    output.write("            },\n")
    output.write("            data: [")
    for item in date_list[::-1] :
        output.write("'%s'," % item)
    output.write("        ]\n")
    output.write("        }\n")
    output.write("    ],\n")
    output.write("    yAxis: [\n")
    output.write("        {\n")
    output.write("            type: 'value',\n")
    output.write("            name: '%s',\n" % Market)
    output.write("            min: 0,\n")
    output.write("            max: %s,\n" % str(max_value_A))
    output.write("            position: 'right',\n")
    output.write("            axisLine: {\n")
    output.write("                lineStyle: {\n")
    output.write("                    color: colors[0]\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            axisLabel: {\n")
    output.write("                formatter: '{value} 원'\n")
    output.write("            }\n")
    output.write("        },\n")
    output.write("        {\n")
    output.write("            type: 'value',\n")
    output.write("            name: '%s',\n" % Dart_Comp_Name)
    output.write("            min: 0,\n")
    output.write("            max: %s,\n" % str(max_value_B))
    output.write("            position: 'left',\n")
    output.write("            axisLine: {\n")
    output.write("                lineStyle: {\n")
    output.write("                    color: colors[1]\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            axisLabel: {\n")
    output.write("                formatter: '{value} 원'\n")
    output.write("            }\n")
    output.write("        }\n")
    output.write("    ],\n")
    output.write("    series: [\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % Market)
    output.write("            type: 'line',\n")
    output.write("              data: [\n")
    for x in closing_list_A[::-1] :
        output.write("'%s'," % x)
    output.write("        ]\n")
    output.write("        },\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % Dart_Comp_Name)
    output.write("            type: 'line',\n")
    output.write("            yAxisIndex: 1,\n")
    output.write("              data: [\n")
    for x in closing_list_B[::-1]  :
        output.write("'%s'," % x)
    output.write("    ]\n")
    output.write("        }\n")
    output.write("    ]\n")
    output.write("};\n")
    output.write("        // use configuration item and data specified to show chart\n")
    output.write("        myChart.setOption(option);\n")
    output.write("    </script>\n")
    output.write("</body>\n")

    # for line in input_2.readlines():
        # output.write(line)

    print("# MAKE HTML Done")

    output.close()

    #read input file
    if platform != 'darwin' :
        with codecs.open(File_Name, 'r', encoding = 'utf8') as file:
            lines = file.read()
        #write output file
        with codecs.open(File_Name, 'w', encoding = 'utf8') as file:
            file.write(lines)

############################################################
if(__name__ == '__main__') :

    Stock_Dictionary = {
        '코드' : ['160580', '138910', '225130', '225130'],
        '종목명' : ['Tiger구리선물', 'KODEX구리선물', 'KINDEX골드선물레버', 'KODEX WTI원유선물'],
        'File_Name' : ['TigerCooper', 'KodexCooper', 'KindexGold', 'KodexWTI'],
    }

    Dart_Market_Name = '코스피'
    # Dart_Market_Name = '코스닥'

    KOSPI_Page = __Last_Page_Numb_Market("KOSPI")
    print("# KOSPI Page : %s" % KOSPI_Page)
    KOSPI_Date, KOSPI_List = __Market_Price("KOSPI", KOSPI_Page)

    # print("# KOSPI DATE")
    # print(len(KOSPI_Date))
    # print(KOSPI_Date)
    # print("# KOSPI List")
    # print(len(KOSPI_List))
    # print(KOSPI_List)

    # KOSDAQ_Page = __Last_Page_Numb_Market("KOSDAQ")
    print("# KOSDAQ Page : %s" % KOSPI_Page)
    # KOSDAQ_Date, KOSDAQ_List = __Market_Price("KOSDAQ", KOSDAQ_Page)

    Stock_Dic = pd.DataFrame(Stock_Dictionary)
    print(Stock_Dic)

    for index, row in Stock_Dic.iterrows():
        print("%s차 FOR문 시작" % index)
        Exp_page = __Last_Page_Numb(row['코드'], 1)
        Dart_Exp_Code = row['코드']
        Comp_Date_List, Comp_Closing_List = __Price_Find(Dart_Exp_Code, Exp_page)

        print("# Comp Date List" )
        print(len(Comp_Date_List))
        print(Comp_Date_List)
        print("# Comp Closing List")
        print(len(Comp_Closing_List))
        print(Comp_Closing_List)

        __New_Make_Chart(Dart_Exp_Code, row['종목명'], Dart_Market_Name, KOSPI_Date, KOSPI_List)

        print("%s차 FOR문 끝" % index)

    print("# Program End")
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.error import HTTPError
from urllib.parse import quote
from matplotlib import pyplot as plt
from sys import platform
import bs4, ssl
import pandas as pd
import re
import datetime, time # for sleep
import requests
import numpy as np
import io
import codecs

import xml.etree.ElementTree as et 

import json
import urllib
import urllib.request
import sys

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                            'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                            'Chrome/23.0.1271.64 Safari/537.11', \
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
            'Accept-Encoding': 'none', \
            'Accept-Language': 'en-US,en;q=0.8', \
            'Connection': 'keep-alive'}

context = ssl._create_unverified_context()

############################################################
def __Last_Page_Find(i, Market) :
    # print("# Last_Page_Find Start")
    if Market == 1 :
        url = "https://finance.naver.com/sise/sise_market_sum.nhn?&page=" + str(i)
    else :
        url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page=" + str(i)
    # print("# print URL")
    # print(url) # ---> 당시의 위치정보를 기반으로 15페이지 가져오는 듯, 어찌 가져오는지는 모르겠음...(2020.05.21)
    # ---> HTTP Error 403: Forbidden
    # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    # req = Request(url=url, headers={'User-Agent': 'Mozilla/5.0', 'referer': 'http://m.naver.com'})

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
                             'AppleWebKit/537.11 (KHTML, like Gecko) ' \
                             'Chrome/23.0.1271.64 Safari/537.11', \
               'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
               'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
               'Accept-Encoding': 'none', \
               'Accept-Language': 'en-US,en;q=0.8', \
               'Connection': 'keep-alive'}
    req = Request(url=url, headers=headers)

    try :
        html = urlopen(req).read()
        # html = urlopen(url, context=context)
    except HTTPError as err:
        if err.code == 404 :
            print("# Error HTTP : 404")
            return
        elif err.code == 403 :
            print("# Error HTTP : 403")
            return
        elif err.code == 500 :
            print("# Error HTTP : 500")
            return
        elif err.code == 505:
            print("# Error HTTP : 505")
            return
        else:
            raise

    bs_object = bs4.BeautifulSoup(html, "html.parser")
    # test = bs_object.find('table' ,attrs={'class':'type_2'})
    # numbersss = test.findAll("td", {"class": "number"})
    # print(numbersss)

    data = []
    table = bs_object.find('table', attrs={'class':'type_2'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    # print("##################")
    # for x in data :
        # print(x)

    # print("##########################################")
    # print(type(data))
    Data_DF = pd.DataFrame(data, columns=['No','회사명','현재가','전일비','등락률','액면가','시가총액','상장주식수','외국인비율','거래량','PER','ROE'])
    # print(type(data_df))
    # print(data_df)
    # print("# Drop NA")
    Data_DF = Data_DF.dropna()
    Data_DF = Data_DF[['No','회사명','ROE','PER','외국인비율','현재가']]
    if Market == 1 :
        Data_DF['Market'] = "코스피"
    else :
        Data_DF['Market'] = "코스닥"

    return Data_DF

############################################################
def __Last_Page_Numb(company_code, dist) :

    if dist == 1 :
        url = "https://finance.naver.com/item/sise_day.nhn?code=" + company_code
    else :
        url = "https://finance.naver.com/research/company_list.nhn?searchType=itemCode&itemCode=" + company_code
    
    print(url)

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' \
        'AppleWebKit/537.11 (KHTML, like Gecko) ' \
        'Chrome/23.0.1271.64 Safari/537.11', \
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', \
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3', \
        'Accept-Encoding': 'none', \
        'Accept-Language': 'en-US,en;q=0.8', \
        'Connection': 'keep-alive'}

    req = Request(url=url, headers=headers)
    html = urlopen(req).read()
    bs_object = bs4.BeautifulSoup(html, "html.parser")
    table = bs_object.find('td', attrs={'class':'pgRR'})
    if table is None :
        print("# Naver Report 없음")
        return 10000

    print(table)

    for x in table.find_all('a') :
        test = x['href']
        active = test.split("=")
        if dist == 1 :
            page_num = active[2]
        else :
            page_num = active[3]

    if dist == 1 :
        print("# Company Page_Number : ", page_num)
    else :
        print("# Report Page_Number : ", page_num)

    return page_num
############################################################
def __Last_Page_Numb_Market(Market) :
    if Market == "KOSPI" :
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=1"
        # print(url)
    elif Market == "KOSDAQ" :
        url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSDAQ&page=1"
        # print(url)
    print("Market : %s" % url)

    html = urlopen(url, context=context)
    bs_object = bs4.BeautifulSoup(html, "html.parser")
    main1_pages = bs_object.find_all("a")
    for a_list in main1_pages:
        if a_list is main1_pages[-1] :
            last_page=a_list.attrs['href']
        else :
            continue
        page_num: object=last_page.split('=')
        print("# Market Page_Number : ", page_num[2])
        return page_num[2]

############################################################
def __Price_Find(company_code, page):
    page = int(page)
    df_list, date_list, closing_list, gap_list, gap_perf  = [], [], [], [], []

    date_list, closing_list = [], []
    for page_num in range(1, page) :
    # for page_num in range(1, 125) : # ---> 1페이지 당 10일, 250일이 년중 평일이라 하면 25페이지가 1년, 125페이지가 5년.
        url= "https://finance.naver.com/item/sise_day.nhn?code=" + company_code + "&page=" + str(page_num)
        print(url)

        req = Request(url=url, headers=headers)

        try :
            html = urlopen(req).read()
            # html = urlopen(url, context=context)
        except HTTPError as err:
            if err.code == 404 :
                print("# Error HTTP : 404")
                return
            elif err.code == 403 :
                print("# Error HTTP : 403")
                return
            elif err.code == 500 :
                print("# Error HTTP : 500")
                return
            elif err.code == 505:
                print("# Error HTTP : 505")
                return
            else:
                raise

        bs_object = bs4.BeautifulSoup(html, "html.parser")
        table = bs_object.find('table', attrs={'class':'type2'})
        table_body = table.find_all('td', attrs={'class':'num'})
        date_body = table.find_all('span', attrs={'class':'tah p10 gray03'})

        for x in date_body :
            # print("x.contents type : %s" % type(x.contents)) # ---> x.contents type : <class 'list'>
            date_list.append(x.contents[0])

        testlist = []
        for tr in table_body :
            td = tr.find_all('span', attrs={'class':'tah p11'})
            # print("for tr type : %s" % type(td))
            for x in td :
                # print("for x in td : %s" % type(x)) # ---> for tr type : <class 'bs4.element.ResultSet'>
                # print("for x.contents in td : %s, %s" % (type(x.contents), x.contents)) # ---> for x.contents in td : <class 'list'>
                # print("for x.contents in td : %s, %s" % (type(x.contents[0]), x.contents[0])) # ---> for x.contents in td : <class 'list'>
                # time.sleep(3)
                # ----> 테이블 중간에 전일비 값이 0이면 table이 엉망이 됨
                # ----> 전일비가 안들어오다가 들어간o
                # print("X Contennts : %s, %s" % (x.contents[0], type(x.contents[0])))
                # print("X Contennts : %s, %s" % (x.contents[0], type(str(x.contents[0]))))
                if str(x.contents[0]) == "0" :
                    continue
                else :
                    testlist.append(x.contents[0])
            
        # print("# TEST LIST")
        # print(testlist)

        for x in range(0, len(testlist)) :
            # print(x, testlist[x])
            if x == 0 :
                closing_list.append(testlist[x])
            else :
                result = x % 5
                if result == 0 :
                    closing_list.append(testlist[x])

    # print("############## date_list ##################")
    # print(date_list)
    # print("############## closing_list ##################")
    # print(closing_list)

    return date_list, closing_list

############################################################
def __Market_Price(company_code, page):
    df_list, date_list, closing_list = [], [], []
    # print("# Market : %s" % company_code)

    print("# Company : %s" % company_code)
    page = int(page)
    for page_num in range(1, page) :
    # for page_num in range(1, 100) :
        url_main = "https://finance.naver.com/sise/sise_index_day.nhn?code=" + company_code + "&page=" + str(page_num)
        # print("# url : %s" % url_main)
        code_data = pd.read_html(url_main, header=0, skiprows=(6,7,8)) # ---> code data로 출력을 생성하면 NaN이 1개라인이 남음, Type은 <class 'list'>
        df_list.append(pd.read_html(url_main)[0]) # ---> List 로 append, 본문[0]과 페이지 출력[1] 부분 중 본문을 출력

    df=pd.concat(df_list)
    df.dropna(subset=["날짜"], inplace=True) # ---> Naver에서 출력되는 가운데 날짜 컬럼 NaN 라인들 삭제처리
    df.rename(columns={'날짜': 'date'}, inplace = True)
    df.rename(columns={'체결가': 'closing'}, inplace = True)
    df.rename(columns={'전일비': 'diff'}, inplace = True)
    df['new_date'] = pd.to_datetime(df['date']) # ---> dateime 형식으로 new_date 추가
    date_list=df.date.values.tolist()
    closing_list=df.closing.values.tolist()
    for x in range(0, len(closing_list)-1):
        closing_list[x] = int(closing_list[x]) # ---> float 에서 int로 변경

    # date_list.reverse()
    # closing_list.reverse()

    return date_list, closing_list

############################################################
def __New_Make_Chart (Dart_Exp_Code, Dart_Comp_Name, Market, Market_Date, Market_List) :

    DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")

    if platform == 'darwin' : # or win32
        DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")
        File_Name = "/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/"+Dart_Comp_Name+".html"
        input_1 = open('/Users/choiyoungmin/PycharmProjects/2020Y/Anal_Comp/New_Echart_Test_1.html', 'r') # Echrt의 머릿글
    else :
        DateTime_Win = datetime.datetime.today().strftime("%Y%m%d")
        File_Name = Dart_Comp_Name+".html"
        input_1 = open('New_Echart_Test_1.html', 'r', encoding='utf-8') # Echrt의 머릿글

    # 아래는 Make_Chart를 위한 조건들
    Comp_Date_List, KOSPI_DATE_List, KOSDAQ_DATE_List = [], [], []
    Comp_Closing_List, KOSPI_List, KOSDAQ_List = [], [], []

    Exp_page = __Last_Page_Numb(Dart_Exp_Code, 1)
    Comp_Date_List, Comp_Closing_List = __Price_Find(Dart_Exp_Code, Exp_page)
    date_list = Comp_Date_List

    print("Market : %s" % Market)

    # if Market == "코스피" :
    #     KOSPI_Page = __Last_Page_Numb_Market("KOSPI")
    #     KOSPI_Date, KOSPI_List = __Market_Price("KOSPI", KOSPI_Page)
    #     closing_list_A = KOSPI_List
    #     closing_list_B = Comp_Closing_List
    # elif Market == "코스닥" :
    #     KOSDAQ_Page = __Last_Page_Numb_Market("KOSDAQ")
    #     KOSDAQ_Date, KOSDAQ_List = __Market_Price("KOSDAQ", KOSDAQ_Page)
    #     closing_list_A = KOSDAQ_List
    #     closing_list_B = Comp_Closing_List

    closing_list_A = Market_List
    closing_list_B = Comp_Closing_List

    n = len(closing_list_B)
    print("date list 0 : %s" % date_list[0])
    print("date leng : %s" % len(date_list))
    print("closing list A-0 : %s " % closing_list_A[0])
    print("closing leng A : %s " % len(closing_list_A))
    print("closong list B-0 : %s " % closing_list_B[0])
    print("closing leng B : %s " % len(closing_list_B))
    del date_list[n:]
    del closing_list_A[n:]
    del closing_list_B[n:]
    print("# After DEL")
    print("date list 0 : %s" % date_list[0])
    print("closing list A-0 : %s " % closing_list_A[0])
    print("closing leng A : %s " % len(closing_list_A))
    print("closong list B-0 : %s " % closing_list_B[0])
    print("closing leng B : %s " % len(closing_list_B))

    output = open(File_Name, 'w+t', encoding='utf-8')

    # ls = [type(item) for item in closing_list_A]
    # print(ls) # ---> int
    # ls = [type(item) for item in closing_list_B]
    # print(ls) # ---> <class 'bs4.element.NavigableString'>
    # 천 단위 , 자리수가 있으면 string이고 없으면 int 로 list 생성되는듯..

    for idx, val in enumerate(closing_list_A) :
        if type(closing_list_A[idx]) == int or type(closing_list_A[idx]) == float :
            continue
        else :
            val = val.replace(",","")
            closing_list_A[idx] = val

    for idx, val in enumerate(closing_list_B) :
        if type(closing_list_B[idx]) == int or type(closing_list_B[idx]) == float :
            continue
        else :
            val = val.replace(",","")
            closing_list_B[idx] = val

    # print(date_list.reverse()) # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x12235bbb0>
    # date_list.reverse() # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x1190a7d00>
    # print(date_list[::-1]) # ---> OK

    # max_value_A = int(max(closing_list_A)) * 2
    # max_value_B = int(max(closing_list_B)) * 2
    numrow_A = [float(x) for x in closing_list_A]
    max_value_A = int(max(numrow_A)) * 1.5
    numrow_B = [float(x) for x in closing_list_B]
    max_value_B = int(max(numrow_B)) * 1.5 # ---> ValueError: could not convert string to float: '11,550'

    print("# MAKE HTML Start")
    for line in input_1.readlines():
        output.write(line)
    output.write("        data: ['%s', '%s']\n" % (Market, Dart_Comp_Name))
    output.write("    },\n")
    output.write("    xAxis: [\n")
    output.write("        {\n")
    output.write("            type: 'category',\n")
    output.write("            axisTick: {\n")
    output.write("                alignWithLabel: true\n")
    output.write("            },\n")
    output.write("            data: [")
    for item in date_list[::-1] :
        output.write("'%s'," % item)
    output.write("        ]\n")
    output.write("        }\n")
    output.write("    ],\n")
    output.write("    yAxis: [\n")
    output.write("        {\n")
    output.write("            type: 'value',\n")
    output.write("            name: '%s',\n" % Market)
    output.write("            min: 0,\n")
    output.write("            max: %s,\n" % str(max_value_A))
    output.write("            position: 'right',\n")
    output.write("            axisLine: {\n")
    output.write("                lineStyle: {\n")
    output.write("                    color: colors[0]\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            axisLabel: {\n")
    output.write("                formatter: '{value} 원'\n")
    output.write("            }\n")
    output.write("        },\n")
    output.write("        {\n")
    output.write("            type: 'value',\n")
    output.write("            name: '%s',\n" % Dart_Comp_Name)
    output.write("            min: 0,\n")
    output.write("            max: %s,\n" % str(max_value_B))
    output.write("            position: 'left',\n")
    output.write("            axisLine: {\n")
    output.write("                lineStyle: {\n")
    output.write("                    color: colors[1]\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            axisLabel: {\n")
    output.write("                formatter: '{value} 원'\n")
    output.write("            }\n")
    output.write("        }\n")
    output.write("    ],\n")
    output.write("    series: [\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % Market)
    output.write("            type: 'line',\n")
    output.write("              data: [\n")
    for x in closing_list_A[::-1] :
        output.write("'%s'," % x)
    output.write("        ]\n")
    output.write("        },\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % Dart_Comp_Name)
    output.write("            type: 'line',\n")
    output.write("            yAxisIndex: 1,\n")
    output.write("              data: [\n")
    for x in closing_list_B[::-1]  :
        output.write("'%s'," % x)
    output.write("    ]\n")
    output.write("        }\n")
    output.write("    ]\n")
    output.write("};\n")
    output.write("        // use configuration item and data specified to show chart\n")
    output.write("        myChart.setOption(option);\n")
    output.write("    </script>\n")
    output.write("</body>\n")

    # for line in input_2.readlines():
        # output.write(line)

    print("# MAKE HTML Done")

    output.close()

    #read input file
    if platform != 'darwin' :
        with codecs.open(File_Name, 'r', encoding = 'utf8') as file:
            lines = file.read()
        #write output file
        with codecs.open(File_Name, 'w', encoding = 'utf8') as file:
            file.write(lines)

############################################################
if(__name__ == '__main__') :

    print("##############################")
    start = time.time()
    print(start)
    
    Stock_Dictionary = {
        '코드' : ['160580', '138910', '225130', '225130'],
        '종목명' : ['Tiger구리선물', 'KODEX구리선물', 'KINDEX골드선물레버', 'KODEX-WTI원유선물'],
        'File_Name' : ['TigerCooper', 'KodexCooper', 'KindexGold', 'KodexWTI'],
    }

    Dart_Market_Name = '코스피'
    # Dart_Market_Name = '코스닥'

    print("##############################")
    print(Dart_Market_Name)
    print("##############################")
    KOSPI_Page = __Last_Page_Numb_Market("KOSPI")
    print("# KOSPI Page : %s" % KOSPI_Page)
    KOSPI_Date, KOSPI_List = __Market_Price("KOSPI", KOSPI_Page)

    # print("# KOSPI DATE")
    # print(len(KOSPI_Date))
    # print(KOSPI_Date)
    # print("# KOSPI List")
    # print(len(KOSPI_List))
    # print(KOSPI_List)

    # KOSDAQ_Page = __Last_Page_Numb_Market("KOSDAQ")
    print("# KOSDAQ Page : %s" % KOSPI_Page)
    # KOSDAQ_Date, KOSDAQ_List = __Market_Price("KOSDAQ", KOSDAQ_Page)

    Stock_Dic = pd.DataFrame(Stock_Dictionary)
    print(Stock_Dic)

    for index, row in Stock_Dic.iterrows():
        print("%s차 FOR문 시작" % index)
        Exp_page = __Last_Page_Numb(row['코드'], 1)
        Dart_Exp_Code = row['코드']
        Comp_Date_List, Comp_Closing_List = __Price_Find(Dart_Exp_Code, Exp_page)

        print("# Comp Date List" )
        print(len(Comp_Date_List))
        print(Comp_Date_List)
        print("# Comp Closing List")
        print(len(Comp_Closing_List))
        print(Comp_Closing_List)

        __New_Make_Chart(Dart_Exp_Code, row['종목명'], Dart_Market_Name, KOSPI_Date, KOSPI_List)

        print("%s차 FOR문 끝" % index)

    print("# Program End")
    print("it took : %s sec", time.time() - start)

Market : https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=1
# Market Page_Number :  1365
# KOSPI Page : 1365
# Company : KOSPI
# KOSDAQ Page : 1365
       코드            종목명    File_Name
0  160580      Tiger구리선물  TigerCooper
1  138910      KODEX구리선물  KodexCooper
2  225130   KINDEX골드선물레버   KindexGold
3  225130  KODEX WTI원유선물     KodexWTI
0차 FOR문 시작
https://finance.naver.com/item/sise_day.nhn?code=160580
<td class="pgRR">
<a href="/item/sise_day.nhn?code=160580&amp;page=210">맨뒤
				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
</td>
# Company Page_Number :  210
https://finance.naver.com/item/sise_day.nhn?code=160580&page=1
https://finance.naver.com/item/sise_day.nhn?code=160580&page=2
https://finance.naver.com/item/sise_day.nhn?code=160580&page=3
https://finance.naver.com/item/sise_day.nhn?code=160580&page=4
https://finance.naver.com/item/sise_day.nhn?code=160580&page=5
https://finance.naver.com/item/sise_day.n

https://finance.naver.com/item/sise_day.nhn?code=160580&page=119
https://finance.naver.com/item/sise_day.nhn?code=160580&page=120
https://finance.naver.com/item/sise_day.nhn?code=160580&page=121
https://finance.naver.com/item/sise_day.nhn?code=160580&page=122
https://finance.naver.com/item/sise_day.nhn?code=160580&page=123
https://finance.naver.com/item/sise_day.nhn?code=160580&page=124
https://finance.naver.com/item/sise_day.nhn?code=160580&page=125
https://finance.naver.com/item/sise_day.nhn?code=160580&page=126
https://finance.naver.com/item/sise_day.nhn?code=160580&page=127
https://finance.naver.com/item/sise_day.nhn?code=160580&page=128
https://finance.naver.com/item/sise_day.nhn?code=160580&page=129
https://finance.naver.com/item/sise_day.nhn?code=160580&page=130
https://finance.naver.com/item/sise_day.nhn?code=160580&page=131
https://finance.naver.com/item/sise_day.nhn?code=160580&page=132
https://finance.naver.com/item/sise_day.nhn?code=160580&page=133
https://finance.naver.com

<td class="pgRR">
<a href="/item/sise_day.nhn?code=160580&amp;page=210">맨뒤
				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
</td>
# Company Page_Number :  210
https://finance.naver.com/item/sise_day.nhn?code=160580&page=1
https://finance.naver.com/item/sise_day.nhn?code=160580&page=2
https://finance.naver.com/item/sise_day.nhn?code=160580&page=3
https://finance.naver.com/item/sise_day.nhn?code=160580&page=4
https://finance.naver.com/item/sise_day.nhn?code=160580&page=5
https://finance.naver.com/item/sise_day.nhn?code=160580&page=6
https://finance.naver.com/item/sise_day.nhn?code=160580&page=7
https://finance.naver.com/item/sise_day.nhn?code=160580&page=8
https://finance.naver.com/item/sise_day.nhn?code=160580&page=9
https://finance.naver.com/item/sise_day.nhn?code=160580&page=10
https://finance.naver.com/item/sise_day.nhn?code=160580&page=11
https://finance.naver.com/item/sise_day.nhn?code=160580&page=12
https://finance.nav

https://finance.naver.com/item/sise_day.nhn?code=160580&page=126
https://finance.naver.com/item/sise_day.nhn?code=160580&page=127
https://finance.naver.com/item/sise_day.nhn?code=160580&page=128
https://finance.naver.com/item/sise_day.nhn?code=160580&page=129
https://finance.naver.com/item/sise_day.nhn?code=160580&page=130
https://finance.naver.com/item/sise_day.nhn?code=160580&page=131
https://finance.naver.com/item/sise_day.nhn?code=160580&page=132
https://finance.naver.com/item/sise_day.nhn?code=160580&page=133
https://finance.naver.com/item/sise_day.nhn?code=160580&page=134
https://finance.naver.com/item/sise_day.nhn?code=160580&page=135
https://finance.naver.com/item/sise_day.nhn?code=160580&page=136
https://finance.naver.com/item/sise_day.nhn?code=160580&page=137
https://finance.naver.com/item/sise_day.nhn?code=160580&page=138
https://finance.naver.com/item/sise_day.nhn?code=160580&page=139
https://finance.naver.com/item/sise_day.nhn?code=160580&page=140
https://finance.naver.com

https://finance.naver.com/item/sise_day.nhn?code=138910&page=35
https://finance.naver.com/item/sise_day.nhn?code=138910&page=36
https://finance.naver.com/item/sise_day.nhn?code=138910&page=37
https://finance.naver.com/item/sise_day.nhn?code=138910&page=38
https://finance.naver.com/item/sise_day.nhn?code=138910&page=39
https://finance.naver.com/item/sise_day.nhn?code=138910&page=40
https://finance.naver.com/item/sise_day.nhn?code=138910&page=41
https://finance.naver.com/item/sise_day.nhn?code=138910&page=42
https://finance.naver.com/item/sise_day.nhn?code=138910&page=43
https://finance.naver.com/item/sise_day.nhn?code=138910&page=44
https://finance.naver.com/item/sise_day.nhn?code=138910&page=45
https://finance.naver.com/item/sise_day.nhn?code=138910&page=46
https://finance.naver.com/item/sise_day.nhn?code=138910&page=47
https://finance.naver.com/item/sise_day.nhn?code=138910&page=48
https://finance.naver.com/item/sise_day.nhn?code=138910&page=49
https://finance.naver.com/item/sise_day.

...

https://finance.naver.com/item/sise_day.nhn?code=138910&page=126
https://finance.naver.com/item/sise_day.nhn?code=138910&page=127
https://finance.naver.com/item/sise_day.nhn?code=138910&page=128
https://finance.naver.com/item/sise_day.nhn?code=138910&page=129
https://finance.naver.com/item/sise_day.nhn?code=138910&page=130
https://finance.naver.com/item/sise_day.nhn?code=138910&page=131
https://finance.naver.com/item/sise_day.nhn?code=138910&page=132
https://finance.naver.com/item/sise_day.nhn?code=138910&page=133
https://finance.naver.com/item/sise_day.nhn?code=138910&page=134
https://finance.naver.com/item/sise_day.nhn?code=138910&page=135
https://finance.naver.com/item/sise_day.nhn?code=138910&page=136
https://finance.naver.com/item/sise_day.nhn?code=138910&page=137
https://finance.naver.com/item/sise_day.nhn?code=138910&page=138
https://finance.naver.com/item/sise_day.nhn?code=138910&page=139
https://finance.naver.com/item/sise_day.nhn?code=138910&page=140
https://finance.naver.com

https://finance.naver.com/item/sise_day.nhn?code=138910&page=253
Market : 코스피
date list 0 : 2021.06.23
date leng : 2530
closing list A-0 : 3276 
closing leng A : 2090 
closong list B-0 : 7,795 
closing leng B : 2530 
# After DEL
date list 0 : 2021.06.23
closing list A-0 : 3276 
closing leng A : 2090 
closong list B-0 : 7,795 
closing leng B : 2530 
# MAKE HTML Start
# MAKE HTML Done
1차 FOR문 끝
2차 FOR문 시작
https://finance.naver.com/item/sise_day.nhn?code=225130
<td class="pgRR">
<a href="/item/sise_day.nhn?code=225130&amp;page=146">맨뒤
				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
</td>
# Company Page_Number :  146
https://finance.naver.com/item/sise_day.nhn?code=225130&page=1
https://finance.naver.com/item/sise_day.nhn?code=225130&page=2
https://finance.naver.com/item/sise_day.nhn?code=225130&page=3
https://finance.naver.com/item/sise_day.nhn?code=225130&page=4
https://finance.naver.com/item/sise_day.nhn?code=225130&page=5


https://finance.naver.com/item/sise_day.nhn?code=225130&page=119
https://finance.naver.com/item/sise_day.nhn?code=225130&page=120
https://finance.naver.com/item/sise_day.nhn?code=225130&page=121
https://finance.naver.com/item/sise_day.nhn?code=225130&page=122
https://finance.naver.com/item/sise_day.nhn?code=225130&page=123
https://finance.naver.com/item/sise_day.nhn?code=225130&page=124
https://finance.naver.com/item/sise_day.nhn?code=225130&page=125
https://finance.naver.com/item/sise_day.nhn?code=225130&page=126
https://finance.naver.com/item/sise_day.nhn?code=225130&page=127
https://finance.naver.com/item/sise_day.nhn?code=225130&page=128
https://finance.naver.com/item/sise_day.nhn?code=225130&page=129
https://finance.naver.com/item/sise_day.nhn?code=225130&page=130
https://finance.naver.com/item/sise_day.nhn?code=225130&page=131
https://finance.naver.com/item/sise_day.nhn?code=225130&page=132
https://finance.naver.com/item/sise_day.nhn?code=225130&page=133
https://finance.naver.com

<td class="pgRR">
<a href="/item/sise_day.nhn?code=225130&amp;page=146">맨뒤
				<img alt="" border="0" height="5" src="https://ssl.pstatic.net/static/n/cmn/bu_pgarRR.gif" width="8"/>
</a>
</td>
# Company Page_Number :  146
https://finance.naver.com/item/sise_day.nhn?code=225130&page=1
https://finance.naver.com/item/sise_day.nhn?code=225130&page=2
https://finance.naver.com/item/sise_day.nhn?code=225130&page=3
https://finance.naver.com/item/sise_day.nhn?code=225130&page=4
https://finance.naver.com/item/sise_day.nhn?code=225130&page=5
https://finance.naver.com/item/sise_day.nhn?code=225130&page=6
https://finance.naver.com/item/sise_day.nhn?code=225130&page=7
https://finance.naver.com/item/sise_day.nhn?code=225130&page=8
https://finance.naver.com/item/sise_day.nhn?code=225130&page=9
https://finance.naver.com/item/sise_day.nhn?code=225130&page=10
https://finance.naver.com/item/sise_day.nhn?code=225130&page=11
https://finance.naver.com/item/sise_day.nhn?code=225130&page=12
https://finance.nav

https://finance.naver.com/item/sise_day.nhn?code=225130&page=126
https://finance.naver.com/item/sise_day.nhn?code=225130&page=127
https://finance.naver.com/item/sise_day.nhn?code=225130&page=128
https://finance.naver.com/item/sise_day.nhn?code=225130&page=129
https://finance.naver.com/item/sise_day.nhn?code=225130&page=130
https://finance.naver.com/item/sise_day.nhn?code=225130&page=131
https://finance.naver.com/item/sise_day.nhn?code=225130&page=132
https://finance.naver.com/item/sise_day.nhn?code=225130&page=133
https://finance.naver.com/item/sise_day.nhn?code=225130&page=134
https://finance.naver.com/item/sise_day.nhn?code=225130&page=135
https://finance.naver.com/item/sise_day.nhn?code=225130&page=136
https://finance.naver.com/item/sise_day.nhn?code=225130&page=137
https://finance.naver.com/item/sise_day.nhn?code=225130&page=138
https://finance.naver.com/item/sise_day.nhn?code=225130&page=139
https://finance.naver.com/item/sise_day.nhn?code=225130&page=140
https://finance.naver.com